# LEA FY 25 Budget Balance Tracking

Using the [tabula-py](https://pypi.org/project/tabula-py/) package to extract data from PDFs for campus budget analysis
- Current Month: **February**
- Next Month: **March**

In [1]:
# importing packages
import tabula 
from tabula import read_pdf
import os
import pandas as pd
import string as str

In [2]:
# setting JAVA_HOME and PATH variables
os.environ['JAVA_HOME'] = '/Users/scipio/Documents/FA_Budget_PDFs/jdk-22.0.2.jdk/Contents/Home'
os.environ['PATH'] = '/Users/scipio/Documents/FA_Budget_PDFs/jdk-22.0.2.jdk/Contents/Home/lib/server'

## JAVA_HOME and PATH

The tabula-py package is a wrapper of [tabula-java](https://github.com/tabulapdf/tabula-java) and requires java on your machine. I recommend installing [OpenJDK](https://jdk.java.net/22/) and setting the JAVA_HOME and PATH enivronment variables as demonstrated above using the os package which creates an instance of the JAVA_HOME and PATH variables. For a more permanent instance set the environment variables in the command prompt (Windows) or terminal (Mac)

- The **JAVA_HOME** variable points to the installation location of OpenJDK
- The **PATH** varible points to the location of the Java Virtual Machine (JVM DLL)

In [3]:
# creating month object for naming protocols
month = 2
current_month = 'February'


# importing data from PDF for each campus
df_es_feb = pd.DataFrame(tabula.read_pdf('/Users/scipio/Documents/FA_Budget_PDFs/ES 02.28.25 Budget.PDF',
                                         pages = 'all',pandas_options = {'header': None})[0])

df_ms_feb = pd.DataFrame(tabula.read_pdf('/Users/scipio/Documents/FA_Budget_PDFs/MS 02.28.25 Budget.PDF', 
                                         pages = 'all',pandas_options = {'header': None})[0])

df_hs_feb = pd.DataFrame(tabula.read_pdf('/Users/scipio/Documents/FA_Budget_PDFs/HS 02.28.25 Budget.PDF',
                                         pages = 'all',pandas_options = {'header': None})[0])


## Data importing protocol

- Import data for the month and name df_campus_month format, e.g. df_es_sept
- Import export from previous month
- Clean data
- Concat cleaned data from previous and current months
- Export


In [4]:
# rename object
rename = {
    0:'Account Title',
    1:'Delete',
    2:'Original Bgt',
    3:'New App/Trnsf',
    4:'Revised Bgt',
    5:'Expenditures',
    6:'Encumbrances',
    7:'Avail Balance',
    8:'% Spent'}

# renaming columns
df_es_feb.rename(columns = rename, inplace = True)
df_ms_feb.rename(columns = rename, inplace = True)
df_hs_feb.rename(columns = rename, inplace = True)

# data cleaning
df_es_feb['Expend. Account #'] = df_es_feb['Account Title'].str.split(' ').str[0]
df_ms_feb['Expend. Account #'] = df_ms_feb['Account Title'].str.split(' ').str[0]
df_hs_feb['Expend. Account #'] = df_hs_feb['Account Title'].str.split(' ').str[0]

# adding month column 
df_es_feb['Month'] = month 
df_ms_feb['Month'] = month 
df_hs_feb['Month'] = month

# adding campus column
df_es_feb['Campus'] = 'ES'
df_ms_feb['Campus'] = 'MS'
df_hs_feb['Campus'] = 'HS'

#dropping irrelevant columns
df_es_feb.drop(labels = ['Delete','% Spent'], axis = 1, inplace = True)
df_ms_feb.drop(labels = ['Delete','% Spent'], axis = 1, inplace = True)
df_hs_feb.drop(labels = ['Delete','% Spent'], axis = 1, inplace = True)

# data casting
df_es_feb['Expenditures'] = df_es_feb['Expenditures'].str.replace(',','').astype('float')
df_ms_feb['Expenditures'] = df_ms_feb['Expenditures'].str.replace(',','').astype('float')
df_hs_feb['Expenditures'] = df_hs_feb['Expenditures'].str.replace(',','').astype('float')


# creating column object for filtering
columns = ['Campus','Month','Original Bgt',
           'New App/Trnsf','Revised Bgt','Expenditures',
           'Avail Balance']

# returning relevant columns
df_es_feb = df_es_feb[df_es_feb['Account Title']=='Grand Totals for fund:11'][columns]#--> filtering using columns
df_ms_feb = df_ms_feb[df_ms_feb['Account Title']=='Grand Totals for fund:11'][columns]#--> filtering using columns
df_hs_feb = df_hs_feb[df_hs_feb['Account Title']=='Grand Totals for fund:11'][columns]#--> filtering using columns

# calculating MoM diff
df_es_feb['Expense Change'] = df_es_feb['Expenditures'].diff()
df_ms_feb['Expense Change'] = df_ms_feb['Expenditures'].diff()
df_hs_feb['Expense Change'] = df_hs_feb['Expenditures'].diff() 

# concating dfs
df_feb_agg = pd.concat([df_es_feb,df_ms_feb,df_hs_feb])

In [5]:
# displaying
df_feb_agg.sort_values(by = ['Campus','Month'])

,Campus,Month,Original Bgt,New App/Trnsf,Revised Bgt,Expenditures,Avail Balance,Expense Change
37,ES,2,"4,515,612.00","(234,870.28)","4,280,741.72",2976540.78,"318,367.41",NaN
43,HS,2,"3,562,807.00","(202,075.00)","3,360,732.00",2211239.66,"133,623.11",NaN
41,MS,2,"2,629,211.00","(115,793.00)","2,513,418.00",1623256.53,"196,943.91",NaN


In [6]:
# object for filtering for export
rel_cols = ['Month','Campus','Expenditures','Expense Change','Avail Balance']

#exporting data - August
df_feb_agg[rel_cols].to_excel('/Users/scipio/Documents/FA_Budget_PDFs/Campus_Budgets_'+current_month+'.xlsx',
                              index = False, header = True )